In [1]:
# %%
# Script written by Maximilien Van Migem 12 June 2025 Ghent
# This script converts the original dataset in to the BIDS format
import os.path as op
from pprint import pprint
import shutil
import glob
import mne
import mne_bids
import numpy as np
import pandas as pd

In [2]:
# %%
# Data paths
data_path = r'C:\Users\mvmigem\Documents\data\project_1'
raw_dir_path = op.join(data_path,'raw_data')
event_dir_path = op.join(data_path,'preprocessed','mastoid_ref_csv','events')
bids_root = op.join(data_path, 'project_1-bids-data')

# Subjects
subjects_list = np.arange(1,27)
subject_ids = subjects_list[subjects_list != 20]

In [3]:
if op.exists(bids_root):
    shutil.rmtree(bids_root)

In [4]:
# %%
# Loop to create a list of raws
raw_list = []
bids_list =[]
event_list = []


gender_mapping = {'Other':0,'Male':1,'Female':2}

for sub in subject_ids:
    
    bid_sub_id = f'{sub:02}'
    # Load eeg
    raw_path = op.join(raw_dir_path,f'sub_{sub}','eeg','*.bdf')
    raw_fname = glob.glob(raw_path) [0]
    raw = mne.io.read_raw_bdf(raw_fname, eog = (), preload=False, verbose=50)
    # Load behaviour
    behav_path = op.join(event_dir_path,f'events_{sub:02}.csv')
    behav = pd.read_csv(behav_path)

    # Select event dict for condition
    if behav['setdown'].isin([1]).any():
        event_id = {
        'start_trial':99, 'pos1/seq':11, 'pos1/seq3':13, 
        'pos2/seq2':22, 'pos2/seq4':24,
        'pos3/seq1':31, 'pos3/seq3':33,
        'pos4/seq2':42, 'pos4/seq4':44,
        }
    elif behav['setdown'].isin([2]).any():
        # Event dict
        event_id = {
            'start_trial':99, 'pos1/seq2':12, 'pos1/seq4':14, 
            'pos2/seq1':21, 'pos2/seq3':23,
            'pos3/seq2':32, 'pos3/seq4':34,
            'pos4/seq1':41, 'pos4/seq3':43,
        }
    events = mne.find_events(raw)

    # event_list.append(events)
    # raw.set_annotations(None)
    # # Add to list
    # raw_list.append(raw)
    # # specify power line frequency, and ref as required by BIDS
    # raw.info['line_freq'] = 50
    # # specify sub info
    # sub_age = int(behav['age'].iloc[0])
    # sub_gender = behav['gender'].iloc[0]
    # mapped_gender = gender_mapping[sub_gender]
    
    # subject_info = {'id':sub,'his_id':f'sub-{sub:02}','age':sub_age,'sex':sub_gender}
    # raw.info['subject_info'] = subject_info
    # # specify task
    # task ='ReversalLearning'
    # # Create BIDS path
    # bids_path = mne_bids.BIDSPath(subject=bid_sub_id,task = task, datatype='eeg', root=bids_root)
    # # Add to list
    # bids_list.append(bids_path)



Trigger channel Status has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
235 events found on stim channel Status
Event IDs: [   80    81    82    83   201 65619]
Trigger channel Status has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
4287 events found on stim channel Status
Event IDs: [   12    14    21    23    32    34    41    43    99   254   255 65536
 65789 65791]
Trigger channel Status has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
4277 events found on stim channel Status
Event IDs: [   12    14    21    23    32    34    41    43    99   255 65536 65635
 65789 65791]
Trigger channel Status has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
4285 events found on stim channel Status
Event IDs: [   11    13    22    24    31    33    42    44    99   254   255 65